# Exploración y manipulación de datos con `pandas`

En la clase anterior aprendimos a operar sobre `DataFrames` construidos a partir de estructuras de datos de Python (diccionario, numpy array)

También podemos usar pandas para explorar y manipular datos tabulares que existen como fichero en nuestro sistema o en un servidor remoto

En este clase veremos 

- como crear un `DataFrame` a partir de archivos CSV o excel
- algunas funciones más avanzadas de `pandas`: `MultiIndex` y `groupBy` 


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from IPython.display import YouTubeVideo
from functools import partial
YouTubeVideo_formato = partial(YouTubeVideo, modestbranding=1, disablekb=0,
                               width=640, height=360, autoplay=0, rel=0, showinfo=0)

display("Versión de pandas "+pd.__version__)

## Importar datos tabulares a partir de archivos CSV 

Un archivo  **CSV** (Comma-Separated Values) es una tabla en formato texto plano cuyas columnas están separadas por comas (u otro delimitador)

**Ejemplo**

Consideremos la base de datos ["Dow Jones Index"](https://archive.ics.uci.edu/ml/datasets/Dow+Jones+Index) del repositorio UCI, la cual se distribuye en formato CSV

```{note}
El Dow Jones es un índice bursatil muy utilizado ya que refleja el comportamiento del mercado accionario norteamericano
```

Descarguémo la base de datos y observemos las primeras cinco lineas

In [ ]:
%%bash
wget -cq https://archive.ics.uci.edu/ml/machine-learning-databases/00312/dow_jones_index.zip
unzip -o dow_jones_index.zip
head -5 dow_jones_index.data

Del archivo CSV podemos ver que cada fila tiene un 

- identificador textual de la acción: `AA`
- una fecha de observación: `1/7/2011`
- un precio de apertura, máximo, mínimo y cierre para la fecha: `$15.82, $16.72, $15.78, $16.42`
- entre otros

También podemos notar algunos aspectos típicos de los archivos CSV

- Las columnas están separadas por comas
- La primera fila del archivo CSV contiene el *header*, es decir los nombres de las columnas
- Las columnas son de tipos distintos: ¿Qué tipos puedes identificar en el ejemplo anterior?

> A continuación veremos como importar y escribir un archivo CSV usando `pandas`

**Función `pd.read_csv`**

Leer un archivo CSV como DataFrame es directo usando la función `read_csv`

A continuación se resaltan los argumentos principales

```python
pd.read_csv(
    filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]], # path completo al archivo CSV
    sep=',', # String o expresión regular que se usará para delimitar las columnas
    header='infer', # Puede ser un int (fila donde está el header) o una lista de de int's
    names=None, # Lista de strings con nombres de columnas (útil si el CSV no tiene header)
    index_col=None, # La columna que se usará como header
    usecols=None, # Lista: subconjunto de columnas que se desean importar (por defecto se importan todas)
    converters=None, # Se explica en detalle más adelante junto a otros argumentos de parsing
    parse_dates=None, # Se explica en detalle más adelante junto a otros argumentos de fecha
    ...
    )
```

Más adelante veremos más argumentos y un ejemplo de uso

**Atributo `to_csv()`**

Podemos crear un archivo CSV desde un DataFrame usando el atributo `to_csv` como se muestra a continuación

```python
    df = pd.DataFrame(data)
    df.to_csv("mis_datos.csv")
```

- Esto  crea un archivo `mis_data.csv` en el directorio actual
- Por defecto guardara las nombres de columna como un header y usará "," como delimitador


## Análisis sintático o *parsing*

En general un archivo de texto plano podría contener

- valores numéricos continuos
- valores numéricos discretos
- fechas
- coordenadas 
- monedas
- direcciones
- etiquetas de texto
- y un largo etcétera

Los programas que leen e importan archivos de texto plano como CSV deben interpretar estos valores y convertirlos al formato más adecuado, por ejemplo

- flotante
- entero
- booleano
- string

Se llama ***parser* o analizador sintático** al programa que analiza los textos y luego 

- filtra y/o completa los textos invalidos
- convierte los datos a un formato estándar

Pandas hace este proceso de forma automática y podemos hacer algunos ajustes usando los argumentos disponibles en `read_csv`

Por ejemplo 


```python
pd.read_csv(
    ...
    dtypes=None # Diccionario donde la llave es el nombre de la columna y el valor el tipo requerido
    na_values=None, # String o lista de strings con valores que serán reconocidos como NaN
    decimal='.', # String que se usará para reconocer el punto decimal
    comment=None, # String, todos las lineas que empiezen con este string serán ignoradas
    converters=None # Se explica a continuación
    ...
    )
```

Si las opciones automáticas no son suficientes se puede hacer *parsing* en base a reglas manualmente creadas usando el argumento `converters`

`converters` recibe un diccionario con "reglas de parseo" con la siguiente sintaxis

```python
    {'nombre de la columna 1': funcion_parseadora1, 
     'nombre de la columna 2': funcion_parseadora2,
     ...
    }
```

Notar que `funcion_parseadoraX` puede ser una función explicita o anómina (lambda)

**Ejemplo**

Los datos de la columna de precio de apertura (open) de "dow_jones_index.data" están formateados como 

`'$15.84'`

que corresponde a un signo dolar seguido de un número real con punto decimal

Para *parsear* este valor debemos escribir una función que 

1. Elimine el signo dolar del string
1. Convierta el resto del string en flotante

Por ejemplo

```python
def remove_dollar(text):
    # return float(x[1:]) # Elimina el primer caracter
    return float(x.strip("$")) # Elimina todos los $ del string
```

Luego agregamos esta función a un diccionario con la llave `open` y se lo entregamos al argumento `converters`, es decir 

```python
parser = {'open': remove_dollar}
```

Se puede lograr lo mismo usando una función anónima, por ejemplo:

```python
parser = {'open': lambda x: float(x.strip("$"))}
```


### Interpretación/parseo de fechas

Un dato textual muy usual en datos tabulares y series de tiempo son las fechas

Sin embargo el formato de fecha puede variar considerablemente entre distintas bases de datos

`pandas` tiene un tipo denominado `Timestamp` el cual se puede construir con la función `pd.to_datetime()` a partir de un string 

Pandas identifica automaticamente fechas y horas en distintos formatos

**Ejemplo**

```python
>>> pd.to_datetime("1/5/2018") # Formato norteamericano Mes/Día/Año 
Timestamp('2018-01-05 00:00:00')

>>> pd.to_datetime("May/1/2018") # También se acepta un string para el mes
Timestamp('2018-05-01 00:00:00')

>>> pd.to_datetime("1st of May of 2018") # También se puede usar una frase "Día del Mes del Año"
Timestamp('2018-05-01 00:00:00')

>>> pd.to_datetime("2018") # Autocompletación por defecto para fechas incompletas
Timestamp('2018-01-01 00:00:00')

>>> pd.to_datetime("14:45") # Si usamos sólo la hora se usa la fecha actual
Timestamp('2020-06-12 14:45:00')

>>> pd.to_datetime("May/1/2018 14:45") # Timestamp completo
Timestamp('2018-05-01 14:45:00')
```

Podemos controlar el parseo de fechas en `read_csv` con los argumentos

```python
pd.read_csv(
    ...
    parse_dates=False # Booleano o lista con las columnas que deben ser interpretadas como fechas
    infer_datetime_format=False, # Inferir una función parseadora de forma automática
    dayfirst=False, # Formato día/mes/año o mes/día/año
    date_parser=None, # Función provista por el usuario que toma un string y retorna TimeStamp
    ...
    )
```

Las fechas/tiempos en formato `TimeStamp` pueden usarse como índices

Esto nos permite recuperar rapidamente todos los eventos dentro de un intervalo de tiempo

### Ejercicio práctico

1. Lea el archivo `dow_jones_index.data` con `pd.read_csv` con las opciones por defecto y estudie el DataFrame resultante
1. Corrija incrementalmente:
    1. Use un conversor para todas las columnas numéricas que empiezan con `$`
    1. Use el argumento `parse_dates` para importar la columna *date* como un `Timestamp` de pandas

En cada paso verifique el tipo de las columnas con el atributo `dtypes`

Con su tabla adecuadamente formateada retorne los valores de apertura (open), cierre (close), mínimo (low) y máximo (high) para las acciones de Alcoa Corp. (AA) entre el Marzo y Junio del 2011


**Solución paso a paso con comentarios:**

In [ ]:
YouTubeVideo_formato('zrBQQQZeGXw')

## Importar datos tabulares a partir de archivos excel

Muchas empresas e instituciones manejan sus datos como hojas de cálculo o *spreadsheets* construidas usando software como Microsoft Excel, Openoffice/Libreoffice calc o Google spreadsheets

`pandas` permite importar como `DataFrame` una hoja de cálculo en formatos `xls, xlsx, xlsm, xlsb, odf` usando  la función [`read_excel`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html)

   
Muchos de los argumentos de `read_csv` están disponibles en `read_excel`, los "nuevos" argumentos son

```python
pd.read_excel(io, # string o path a la hoja de cálculo
              sheet_name=0, # Entero, string o lista, especifica la(s) hoja (s) que vamos a importar
              ...
             )
```

**Nota** 

Para trabajar con archivos `excel` se requieren algunas librerías adicionales las cuales puede instalarse facilmente con conda

    conda install xlrd openpyxl
    
    
**Ejemplo**

Consideremos los siguientes datos del censo chileno de 2017 en formato Excel de donde importaremos datos de vivienda por comuna

Esto corresponde a la segunda hoja (`sheet_name=1`) y en particular las columnas de 1 a 20

Importemos la planilla e inspecciones sus primeras filas

In [ ]:
!wget -cq http://www.censo2017.cl/wp-content/uploads/2017/12/Cantidad-de-Viviendas-por-Tipo.xlsx
df = pd.read_excel("Cantidad-de-Viviendas-por-Tipo.xlsx", 
                   sheet_name=1, # Importamos la segunda hoja (vivienda)
                   usecols=list(range(1, 20)), # Importamos las columnas 1 a 20
                   header=1, # El header está en la segunda fila
                   skiprows=[2], # Eliminamos la fila 2 ya que es invalida
                   index_col='ORDEN' # Usaremos la columna orden como índice
                  ).dropna() # Eliminamos las filas con NaN

display(df.head())

## Manipulando índices y multi-índices

Estudiando la tabla anterior notamos que tiene una estructura jerárquica

> REGION, PROVINCIA, COMUNA

Podemos representar este tipo de estructuras en pandas usando un `MultiIndex` 

**Creación y remoción de índices**

Para asignar un índice a un DataFrame que ya está creado podemos usar el atributo

```python
df.set_index(keys, # Una etiqueta o una lista de etiquetas que serán los nuevos índices
             drop=True, # Eliminar las columnas que pasarán a ser índices
             inplace=False, # Retornar un nuevo dataframe o modificar df
             ...
            )
```

- Si keys es una etiqueta crearemos un índice regular
- Si keys es una lista crearemos un `MultiIndex`

Si queremos que nuestro índice o multi-índice vuelva a convertirse en columna podemos usar el atributo

```python
df.reset_index(level = None, # Permite especificar cuantos niveles de índices se removeran
               drop: bool = False, # Si los índices se deben eliminar o agregar como columnas
               inplace: bool = False,  # Retornar un nuevo dataframe o modificar df
               ...
               )
```

**Ejemplo**

Crearemos un `MultiIndex` para la tabla del censo. Usaremos dos niveles de jerarquía, el superior para región y el inferior para provincia. Esto se logra con

```python
df.set_index(["NOMBRE REGIÓN", "NOMBRE PROVINCIA"], inplace=True) 
```

Si inspeccionamos el objeto `df.index` veremos lo siguiente:

```python
MultiIndex([(                  'ARICA Y PARINACOTA',             'ARICA'),
            (                  'ARICA Y PARINACOTA',             'ARICA'),
            (                  'ARICA Y PARINACOTA',        'PARINACOTA'),
            ...
```


**Ejemplos de *Slicing* con `MultiIndex`**

Para recuperar un elemento de un DataFrame con `MultiIndex` podemos indexar usando una tupla especificando cada uno de los niveles de índices

Por ejemplo para recuperar una fila en particular usamos

```python
df.loc[("LOS RÍOS", "VALDIVIA", "VALDIVIA")]
```
Lo cual retorna la comuna de Valdivia 

Si queremos recuperar un conjunto de elementos podemos usar

```python
df.loc[("LOS RÍOS", "VALDIVIA")]
```
Lo cual retorna todas las comunas de la provincia de Valdivia

También podríamos usar

```python
df.loc[("LOS RÍOS")] 
```
Lo cual retorna todas las comunas de región de los rios

Para hacer *slices* o `fancy indexing`, lo más simple es usar el objeto [`IndexSlice`](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.IndexSlice.html) 

Por ejemplo para recuperar dos filas usamos

```python
idx = pd.IndexSlice
df.loc[idx[:, :, ["VALDIVIA", "OSORNO"]], :] 
```
Lo cual retorna las comunas de Valdivia y Osorno

Para recuperar subconjuntos arbitrarios de filas en base al índice jerárquico podemos usar 

```python
idx = pd.IndexSlice
df.loc[idx[:, ["LLANQUIHUE", "PALENA"], : ], :]
```

Lo cual retorna las comunas pertenecientes a las provincias de Llanquihue y Palena

### Ejercicio práctico

- Asigne un MultiIndex al DataFrame de datos de vivienda del censo
    - Use como primer nivel la etiqueta "NOMBRE REGION"
    - Use como segundo nivel la etiqueta "NOMBRE PROVINCIA" 
    - Use como tercer nivel la etiqueta "NOMBRE COMUNA"
- Use `loc` para seleccionar
    - las comunas de la región de "LOS RÍOS"
    - las comunas de las provincias de "RANCO" y "OSORNO"
    - las comunas "VALDIVIA" y "FRUTILLAR"
- Selecciona las comunas de la provincia de "VALDIVIA" y usa una reducción suma para encontrar el número de viviendas totales de cada tipo

**Solución paso a paso con comentarios**

In [ ]:
YouTubeVideo_formato('bWjB4089EbA')

## Patrón *Split-Apply-Combine* en DataFrames

Digamos que queremos obtener los totales de todos los tipos de vivienda a nivel de provincia

Si asignamos "NOMBRE PROVINCIA" como índice podríamos usar

```python
result = []
for provincia in df.index.unique():    
    sub_df = df.loc[provincia, col_mask]
    if sub_df.ndim>1:    
        result.append(df.loc[provincia, col_mask].sum())
    else: # No hacer reducción suma si la provincia tiene una sola comuna
        result.append(df.loc[provincia, col_mask])
pd.DataFrame(result, columns=col_mask, index=df.index.unique())
```
que obtiene el resultado deseado, pero es ineficiente y bastante engorroso

El ejemplo anterior representa un patrón de "operaciones condicionadas por llave" que es muy utilizado en bases de datos y se conoce como *split-apply-combine*

<img src="img/groupby.svg">

Donde

- *split*: Divide los datos según una **llave**
- *apply*: Realiza una función sobre cada grupo
- *combine*: Mezcla el resultado en un nuevo dataframe donde la **llave** se convierte en el índice

En el ejemplo anterior 

- *split*: Crea subconjuntos con las comunas de cada provincia
- *apply*: Hace una reducción suma en las columnas de viviendas
- *combine*: Construye un nuevo dataframe con los resultados donde la llave son las provincias 

**Atributo `groupby`**

El patrón *split-apply-combine* está implementado de forma muy eficiente en `pandas` a través del atributo [`groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) 

Los argumentos básicos de `groupby` son

```python
df.groupby(by=None, # Columna o lista de columnas con se hace el split
           axis=0, # Dividir en filas (0) o en columnas (1)
           as_index: bool = True, # Retornar las etiquetas de cada grupo como índice
           sort: bool = True, # Retornar las etiquetas de grupo ordenadas alfabeticamente
           ...
          )    
```

Notemos que `groupby` actua como un iterador

```python
for (region, sub_df) in df.groupby('NOMBRE REGIÓN'):
    display(region, # La etiqueta
            sub_df  # El dataframe con las filas que comparten esa etiqueta
           )
```

La función que se ejecuta a cada grupo en el paso *apply* es un atributo de `groupby`, existen cuatro atributos

- `aggregate`: Para hacer reducciones
- `filter`: Para eliminar filas
- `transform`: Para modificar filas
- `apply`: Función flexible que puede combinar lo que hace `aggregate` y `transform` 

A continuación revisaremos las primeras tres en detalle

**Reducción con `aggregate`**

La sintaxis básica de este atributo es

```python
# Para aplicar la misma función a todos las columnas
df.groupby(by=llave).aggregate(funcion1) 
# Para aplicar varias funciones a todos las columnas
df.groupby(by=llave).aggregate([funcion1, funcion2, ...]) 
# Para aplicar funciones específicas a columnas específicas
df.groupby(by=llave).aggregate({columna1: funcion1, columna2: funcion2}) 
```
Las funciones debe entregar un sólo valor por cada columna del grupo

En general las reducciones se usan para hacer resumenes, por ejemplo sumas, promedios o varianzas

**Ejemplo**

Podemos encontrar los totales de vivienda por provincia en una sola linea usando

```python
df.groupby(by="NOMBRE PROVINCIA", sort=False).aggregate(np.sum)
```

O usando el alias

```python
df.groupby(by="NOMBRE PROVINCIA", sort=False).sum() 
```

**Filtrado con `filter`**

La sintaxis básica de este atributo es

```python
df.groupby(by=llave).filter(funcion)
```

La función debe retornar `True` o `False`

El resultado es un nuevo DataFrame con todos los grupos que "pasaron el filtro"

En general este atributo se usa para eliminar/descartar grupos de filas (drop)

**Modificando el `DataFrame` con `transform`**

La sintaxis básica de este atributo es

```python
df.groupby(by=llave).transform(funcion)
```

La función debe retornar un dataframe con la misma dimensión y tamaño que el original y se aplica columna a columna

La función puede ser explicita o anónima (lambda)

Un uso típico de este atributo es el reescalamiento/normalización a nivel de grupo 

### Ejercicio práctico

Considere las columnas de "viviendas particulares ocupadas con moradores presentes" ($V_1$) y "viviendas particulares ocupadas con moradores ausentes" ($V_2$)

1. Cree un MultiIndex equivalente al del ejercicio anterior
1. Realice una reducción promedio y desviación estándar de cada región
1. Use un filtro para encontrar las comunas "más responsables", es decir aquellas donde $\frac{V_1}{V_1 + V_2} > 0.98$
1. Use una transformación para describir las columnas $V_1$ y $V_2$ como porcentajes a nivel regional

**Solución paso a paso con comentarios**

In [ ]:
YouTubeVideo_formato('BopVrkZKNtw')